# Project 3 - Richardson

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [185]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np
from sklearn.utils.extmath import cartesian #modules=sklearn.utils.extmath is the 'module path'
import os


In [186]:
df_raw = pd.read_csv("/Users/squirrel/Documents/project03/admissions.csv")
df = df_raw.dropna() 
df.head(10)

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0
5,1,760.0,3.00,2.0
6,1,560.0,2.98,1.0
7,0,400.0,3.08,2.0
8,1,540.0,3.39,3.0
9,0,700.0,3.92,2.0


In [187]:
df.tail(5)

,admit,gre,gpa,prestige
395,0,620.0,4.00,2.0
396,0,560.0,3.04,3.0
397,0,460.0,2.63,2.0
398,0,700.0,3.65,2.0
399,0,600.0,3.89,3.0


In [188]:
df.describe()

,admit,gre,gpa,prestige
count,397.000000,397.000000,397.000000,397.000000
mean,0.317380,587.858942,3.392242,2.488665
std,0.466044,115.717787,0.380208,0.947083
min,0.000000,220.000000,2.260000,1.000000
25%,0.000000,520.000000,3.130000,2.000000
50%,0.000000,580.000000,3.400000,2.000000
75%,1.000000,660.000000,3.670000,3.000000
max,1.000000,800.000000,4.000000,4.000000


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [189]:
# frequency table for prestige and whether or not someone was admitted
#df.describe()
ct00 = pd.crosstab(index=df_raw["prestige"],  # Make a crosstab
                              columns="count") 

In [190]:
ct00

col_0,count
prestige,
1.0,61
2.0,150
3.0,121
4.0,67


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [191]:
dummy_ranks = pd.get_dummies(df_raw['prestige'], prefix='prestige')

In [192]:
dummy_ranks.head(10)

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1
5,0,1,0,0
6,1,0,0,0
7,0,1,0,0
8,0,0,1,0
9,0,1,0,0


#### 2.2 When modeling our class variables, how many do we need? 



Answer: 3

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [193]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
handCalc.head()



/Users/squirrel/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


,admit,gre,gpa,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0,0,1,0
1,1,660.0,3.67,0,0,1,0
2,1,800.0,4.00,1,0,0,0
3,1,640.0,3.19,0,0,0,1
4,0,520.0,2.93,0,0,0,1


In [194]:
pd.crosstab(handCalc.admit,handCalc['prestige_1.0'])

prestige_1.0,0,1
admit,,
0,243,28
1,93,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [195]:
33/28

1.1785714285714286

In [196]:
#predictors=['prestige_1.0']
#lm = LogisticRegression()
#lm.fit(ct01[predictors], df['admit'])

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [197]:
93/243

0.38271604938271603

#### 3.3 Calculate the odds ratio

In [198]:
(33/28)/(93/243)

3.079493087557604

#### 3.4 Write this finding in a sentenance: 

Answer: The odds of admittance given attendance at a Tier 1 school is 3.079 greater than those who did not attend a Tier 1 school.

#### 3.5 Print the cross tab for prestige_4

In [199]:
pd.crosstab(handCalc.admit,handCalc['prestige_4.0'])

prestige_4.0,0,1
admit,,
0,216,55
1,114,12


#### 3.6 Calculate the OR 

In [200]:
(12/55)/(114/216)

0.4133971291866028

#### 3.7 Write this finding in a sentence

Answer: The odds of being admitted from a Tier 4 school is less than half (.4134 OR) than someone who graduated from a Tier 1 school.

## Part 4. Analysis

In [201]:
df_raw = pd.read_csv("/Users/squirrel/Documents/project03/admissions.csv")
df = df_raw.dropna() 
df.head(10)

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0
5,1,760.0,3.00,2.0
6,1,560.0,2.98,1.0
7,0,400.0,3.08,2.0
8,1,540.0,3.39,3.0
9,0,700.0,3.92,2.0


In [202]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2.0':])
data.head()

,admit,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0,1,0
1,1,660.0,3.67,0,1,0
2,1,800.0,4.00,0,0,0
3,1,640.0,3.19,0,0,1
4,0,520.0,2.93,0,0,1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [203]:
# manually add the intercept
data['intercept'] = 1.0      #'data' was defined in step [59] above

#### 4.1 Set the covariates to a variable called train_cols

In [223]:
train_cols = ['gre', 'gpa','prestige_2.0','prestige_3.0','prestige_4.0', 'intercept']

#### 4.2 Fit the model

In [224]:
logit = sm.Logit(data['admit'], data[train_cols])

In [225]:
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [226]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Thu, 24 Aug 2017   Pseudo R-squ.:                 0.08166
Time:                        20:18:27   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa              0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2.0    -0.6801      0.317     -2.146      0.032      -1.301      -0.059
prestige_3.0    -1.3387      0.345     -3.882      0.000      -2.015      -0.663
prestige_4.0    -1.5534      0.417     -3.721      0.000      -2.372      -0.735
intercept       -3.8769      1.142     -3.393      0.001      -6.116      -1.638
================================================================================
"""

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [227]:
parameters = result.params #method params calculating parameters from the result set
conf = result.conf_int() #generates the confidence intervals, defaults to 2.5% and 97.5%?
conf['OR'] = parameters #generates the confidence ORs from parameters
conf.columns = ['2.5%', '97.5%', 'OR'] #names the columns
np.exp(conf)  #the exponential of each of the coefficients generates the odds ratios

,2.5%,97.5%,OR
gre,1.000074,1.004372,1.002221
gpa,1.136120,4.183113,2.180027
prestige_2.0,0.272168,0.942767,0.506548
prestige_3.0,0.133377,0.515419,0.262192
prestige_4.0,0.093329,0.479411,0.211525
intercept,0.002207,0.194440,0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: The odds of being admitted when applying from a 2nd Tier school drops to 38% (.384342). We can be highly confident in this result given that 1.0 does not fall within the CIs.

#### 4.6 Interpret the OR of GPA

Answer: The odds of being admitted based on GPA is 87% (.876073). However, we cannot be confident in this result given 1.0 falls within the CI.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [228]:
#This makes zero sense to me

#### 5.1 Recreate the dummy variables

In [229]:
# recreate the dummy variables

# keep only what we need for making predictions

dummy_ranks = pd.get_dummies(df_raw['prestige'], prefix='prestige')

In [230]:
dummy_ranks

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1
5,0,1,0,0
6,1,0,0,0
7,0,1,0,0
8,0,0,1,0
9,0,1,0,0


#### 5.2 Make predictions on the enumerated dataset

In [231]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_ranks = pd.get_dummies(combos['prestige'], prefix='prestige')
dummy_ranks.columns = ['prestige_1.0', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0']

# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1.0':])

# make predictions on the enumerated dataset
combos['admit_pred'] = result.predict(combos[train_cols])

combos.head()
#    gre       gpa  prestige  intercept  prestige_2  prestige_3  prestige_4  admit_pred
# 0  220  2.260000         1          1           0           0           0    0.157801
# 1  220  2.260000         2          1           1           0           0    0.087056
# 2  220  2.260000         3          1           0           1           0    0.046758
# 3  220  2.260000         4          1           0           0           1    0.038194
# 4  220  2.453333         1          1           0           0           0    0.179574

/Users/squirrel/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,gre,gpa,prestige,intercept,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0,admit_pred
0,220.0,2.260000,1.0,1.0,1,0,0,0,0.164173
1,220.0,2.260000,2.0,1.0,0,1,0,0,0.090492
2,220.0,2.260000,3.0,1.0,0,0,1,0,0.048977
3,220.0,2.260000,4.0,1.0,0,0,0,1,0.039890
4,220.0,2.453333,1.0,1.0,1,0,0,0,0.185907


In [232]:
combos.tail()

,gre,gpa,prestige,intercept,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0,admit_pred
395,800.0,3.806667,4.0,1.0,0,0,0,1,0.334286
396,800.0,4.000000,1.0,1.0,1,0,0,0,0.734040
397,800.0,4.000000,2.0,1.0,0,1,0,0,0.582995
398,800.0,4.000000,3.0,1.0,0,0,1,0,0.419833
399,800.0,4.000000,4.0,1.0,0,0,0,1,0.368608


#### 5.3 Interpret findings for the last 4 observations

Answer: GRE and GPA being even (800 and 4.0, respectively) the predicted probability for an applicant from a Tier 1 school is almost double that of an applicant from a Tier 4 school (.734 vs. .368).

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.